In [10]:
from Bio.PDB import PDBParser,DSSP

import numpy as np

from molecular_data import crgbaDSSP, restype, colorrgba, vrad, resdict

In [32]:
#Function to define the center of the point cloud
def centroid(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_z = np.sum(arr[:, 2])
    return sum_x/length, sum_y/length, sum_z/length

class Canvas(object):
    
    visualization_modes = ['cpk','backbone','aminoacid','dssp']
    
    def __init__(self, pdbdata, mode='cpk'):
        
        #Analyze pdb file
        self.parser = PDBParser(QUIET=True, PERMISSIVE=True)
        self.structure = self.parser.get_structure('model',pdbdata)
        
        #DSSP prediction
        self.model = self.structure[0]
        self.dssp = DSSP(self.model, pdbdata)
        
        #Mode selection
        if mode not in Canvas.visualization_modes:
            raise Exception('Not recognized visualization mode %s' % mode)
        self.mode = mode
        
    def atom_information(self):
        
        """Determines the coordinates, colors and sizes of the atoms depending on the mode"""
        
        if self.mode == 'cpk':
            #list of atoms
            self.atoms = [atom for atom in self.structure.get_atoms()]
            #atom coordinates
            self.coordinates = [atom.coord for atom in self.atoms]
            self.center = centroid(self.coordinates)
            self.coordinates -= self.center
            #atom color
            self.color = [colorrgba(atom.get_id()) for atom in self.atoms]
            #atom radius
            self.radius = [vrad(atom.get_id() for atom in self.atoms)]
            
        elif self.mode == 'aminoacid':
            #list of atoms
            self.atoms = [atom for atom in self.structure.get_atoms()]
            #atom coordinates
            self.coordinates = [atom.coord for atom in self.atoms]
            self.center = centroid(self.coordinates)
            self.coordinates -= self.center
            #atom color
            self.color = [colorrgba(restype(atom.get_parent().resname)) for atom in self.atoms]
            #atom radius
            self.radius = [vrad(atom.get_id() for atom in self.atoms)]
            
        elif self.mode == 'backbone':
            #list of atoms
            self.atoms = [atom for atom in self.structure.get_atoms() if atom.get_name() == 'CA' or atom.get_name() == 'N']
            #atom coordinates
            self.coordinates = [atom.coord for atom in self.atoms]
            self.center = centroid(self.coordinates)
            self.coordinates -= self.center
            #atom color
            self.color = []
            self.chains = []
            for chain in self.structure.get_chains():
                self.chains.append(chain)
                self.chain_length = len([atom for atom in chain.get_atoms() if atom.get_name() =='CA' or atom.get_name() =='N'])
                self.chain_color = np.append(np.random.rand(1,3),[1.0])
                self.color.append(np.tile(self.chain_color,(self.chain_length,1)))
            if len(self.chains)>1:
                self.color = np.concatenate(self.color)
            #atom radius
            self.radius = [vrad(atom.get_id() for atom in self.atoms)]
                
        elif self.mode == 'dssp':
            #list of atoms
            self.atoms = [atom for atom in self.structure.get_atoms() if atom.get_name() == 'CA' or atom.get_name() == 'N']
            #atom coordinates
            self.coordinates = [atom.coord for atom in self.atoms]
            self.center = centroid(self.coordinates)
            self.coordinates -= self.center
            #atom color
            self.struct3 = [self.dssp[key][2] for key in list(self.dssp.keys())]
            self.residues = [residue for residue in self.structure.get_residues() if residue.get_resname() in resdict.keys()]
            self.color = []
            for i in range(len(self.struct3)):
                self.dsspcolor = crgbaDSSP(self.struct3[i])
                self.natoms = len([atom for atom in self.residues[i] if atom.get_name() =='CA' or atom.get_name() == 'N'])
                self.color.append(np.tile(self.dsspcolor,(self.natoms,1)))
            if len(self.struct3)>1:
                self.color = np.concatenate(self.color)
            #atom radius
            self.radius = [vrad(atom.get_id() for atom in self.atoms)]
            
    
    

In [4]:
#Desglosamos archivo PDB
pdbdata = 'data/1yd9.pdb'
parser = PDBParser(QUIET=True,PERMISSIVE=True)
structure = parser.get_structure('model', pdbdata)
        
#Hacemos la prediccion DSSP
model = structure[0]
dssp = DSSP(model, pdbdata)

atoms = [atom for atom in structure.get_atoms() if atom.get_name() == 'CA' or atom.get_name() == 'N']